In [1]:
from   symcircuit.symbolic_solver.domains    import Circuit, ExperimentResult
from   symcircuit.symbolic_solver.solver     import Circuit_Solver, Impedance_Analyzer
from   symcircuit.demo.multiple_feedback     import Voltage_Mode_Multiple_Feedback as VMMF
import sympy

# Manual Runs

In [2]:
circuit = VMMF.circuit
circuit.hasSolution()

True

In [3]:
circuit.impedances

[Z_1, Z_2, Z_3, Z_4, Z_5]

In [4]:
print(circuit.nodal_equations)

[Eq(-Va/Z_4 - Va/Z_3 + (V2 - Va)/Z_2 + (V1 - Va)/Z_1, 0), Eq(Va/Z_3, -V2/Z_5)]


In [5]:
circuit.solve_for

[V1, V2, Va]

## Circuit Solver

In [6]:
solver = Circuit_Solver(circuit=circuit, _output= sympy.symbols("V2 0"), _input=sympy.symbols("V1 0"), transmissionMatrixType="NA")

In [7]:
solver.impedances

[Z_1, Z_2, Z_3, Z_4, Z_5]

In [8]:
solver.impedancesToDisconnect

[Z_1, Z_2, Z_3, Z_4, Z_5]

In [9]:
solver.solve()

====Solving the Circuit====
--- Computing the possible impedance connections for [Z_1, Z_2, Z_3, Z_4, Z_5] ---
--- Impedance connections stored in Circuit_Solver.impedanceConnections---
 ----------------------------
1 - Solving for (V2 - 0) / (V1 - 0)
2 - Intermediate Variables: [V1, V2, Va]
 ----------------------------
3 - solved the base transfer function (symbolic [T])
-- Single-ended input/output
processing combo size 1
processing combo size 2
processing combo size 3
processing combo size 4
processing combo size 5
=====*Circuit Solved*=====


In [10]:
symcircuit_sol = solver.baseHs
symcircuit_sol

-Z_2*Z_4*Z_5/(Z_1*Z_2*Z_3 + Z_1*Z_2*Z_4 + Z_1*Z_3*Z_4 + Z_1*Z_4*Z_5 + Z_2*Z_3*Z_4)

In [11]:
Y1, Y2, Y3, Y4, Y5 = sympy.symbols("Y1, Y2, Y3, Y4, Y5")
Z1, Z2, Z3, Z4, Z5 = sympy.symbols("Z1, Z2, Z3, Z4, Z5")

tf_from_the_original_paper = -(Y1*Y3)/((Y1+Y2+Y3+Y4)*Y5 + Y2*Y3)
tf_from_the_original_paper

-Y1*Y3/(Y2*Y3 + Y5*(Y1 + Y2 + Y3 + Y4))

In [12]:
new_form = tf_from_the_original_paper.subs({
    Y1 : 1/Z1,
    Y2 : 1/Z2,
    Y3 : 1/Z3,
    Y4 : 1/Z4,
    Y5 : 1/Z5,
})
new_form = new_form.simplify()
new_form

-Z2*Z4*Z5/(Z1*Z2*Z3 + Z1*Z2*Z4 + Z1*Z3*Z4 + Z1*Z4*Z5 + Z2*Z3*Z4)

## Impedance Analyzer

In [13]:
analysis = Impedance_Analyzer("TEST_VMMF", circuit_solver=solver)
print(analysis.isCircuitSolved())
analysis

True


In [14]:
for block in analysis.impedance_blocks:
    print(f"{block.name} - {block.symbol} - allowed connections {block.allowedConnections}")

1 - Z_1 - allowed connections [R_1, 1/(C_1*s), R_1 + 1/(C_1*s), R_1/(C_1*R_1*s + 1)]
2 - Z_2 - allowed connections [R_2, 1/(C_2*s), R_2 + 1/(C_2*s), R_2/(C_2*R_2*s + 1)]
3 - Z_3 - allowed connections [R_3, 1/(C_3*s), R_3 + 1/(C_3*s), R_3/(C_3*R_3*s + 1)]
4 - Z_4 - allowed connections [R_4, 1/(C_4*s), R_4 + 1/(C_4*s), R_4/(C_4*R_4*s + 1)]
5 - Z_5 - allowed connections [R_5, 1/(C_5*s), R_5 + 1/(C_5*s), R_5/(C_5*R_5*s + 1)]


In [15]:
analysis.computeTFs(comboKey="Z1_Z2_Z3_Z4_Z5")

combo key = Z1_Z2_Z3_Z4_Z5
Zcombos


Getting the TFs (CG): 100%|██████████| 1024/1024 [00:10<00:00, 99.79combo/s]

Number of transfer functions found: 1024


In [16]:
analysis.classifier.classifyBiQuadFilters()

Computing Biquad filter parameters: 100%|██████████| 1024/1024 [00:18<00:00, 54.55filter/s]


In [17]:
analysis.classifier.classifications

[FilterClassification(zCombo=(R_1, R_2, R_3, R_4, R_5), transferFunc=-R_2*R_4*R_5/(R_1*R_2*R_3 + R_1*R_2*R_4 + R_1*R_3*R_4 + R_1*R_4*R_5 + R_2*R_3*R_4), valid=False, fType=INVALID-ORDER, parameters=None),
 FilterClassification(zCombo=(R_1, R_2, R_3, R_4, 1/(C_5*s)), transferFunc=-R_2*R_4/(R_1*R_4 + s*(C_5*R_1*R_2*R_3 + C_5*R_1*R_2*R_4 + C_5*R_1*R_3*R_4 + C_5*R_2*R_3*R_4)), valid=False, fType=INVALID-ORDER, parameters=None),
 FilterClassification(zCombo=(R_1, R_2, R_3, R_4, R_5 + 1/(C_5*s)), transferFunc=(-C_5*R_2*R_4*R_5*s - R_2*R_4)/(R_1*R_4 + s*(C_5*R_1*R_2*R_3 + C_5*R_1*R_2*R_4 + C_5*R_1*R_3*R_4 + C_5*R_1*R_4*R_5 + C_5*R_2*R_3*R_4)), valid=False, fType=INVALID-ORDER, parameters=None),
 FilterClassification(zCombo=(R_1, R_2, R_3, R_4, R_5/(C_5*R_5*s + 1)), transferFunc=-R_2*R_4*R_5/(R_1*R_2*R_3 + R_1*R_2*R_4 + R_1*R_3*R_4 + R_1*R_4*R_5 + R_2*R_3*R_4 + s*(C_5*R_1*R_2*R_3*R_5 + C_5*R_1*R_2*R_4*R_5 + C_5*R_1*R_3*R_4*R_5 + C_5*R_2*R_3*R_4*R_5)), valid=False, fType=INVALID-ORDER, paramete

In [18]:
classifications = analysis.classifier.summarizeFilterType()

HP : 25
BP : 70
LP : 25
BS : 0
GE : 0
AP : 0
INVALID-NUMER : 192
INVALID-WZ : 86
INVALID-ORDER : 626
PolynomialError : 0


In [19]:
analysis.reportSummary("VMMF_BiQuad", "Z1_Z2_Z3_Z4_Z5")
analysis.compilePDF()

LaTeX report generated and saved to: Runs/TEST_VMMF/VMMF_BiQuad_Z1_Z2_Z3_Z4_Z5/VMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_summary.tex

=== Compiling the reports to PDF ===
----** compiling Runs/TEST_VMMF/VMMF_BiQuad_Z1_Z2_Z3_Z4_Z5/VMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_summary.tex **----
PDF generated: Runs/TEST_VMMF/VMMF_BiQuad_Z1_Z2_Z3_Z4_Z5/VMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_summary.pdf
=== Compiling DONE ===



In [20]:
analysis.classifier.classifyFilter(filterOrder="FirstOrder")

Computing filter parameters for FirstOrder: 100%|██████████| 1024/1024 [00:05<00:00, 200.69filter/s]


In [21]:
classifications = analysis.classifier.summarizeFilterType()

HP : 25
BP : 38
LP : 25
BS : 0
GE : 0
AP : 0
INVALID-NUMER : 0
INVALID-WZ : 0
INVALID-ORDER : 936
PolynomialError : 0


In [22]:
analysis.reportSummary("VMMF_FirstOrder", "Z1_Z2_Z3_Z4_Z5")
analysis.compilePDF()

LaTeX report generated and saved to: Runs/TEST_VMMF/VMMF_FirstOrder_Z1_Z2_Z3_Z4_Z5/VMMF_FirstOrder_Z1_Z2_Z3_Z4_Z5_summary.tex

=== Compiling the reports to PDF ===
----** compiling Runs/TEST_VMMF/VMMF_BiQuad_Z1_Z2_Z3_Z4_Z5/VMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_summary.tex **----
PDF generated: Runs/TEST_VMMF/VMMF_BiQuad_Z1_Z2_Z3_Z4_Z5/VMMF_BiQuad_Z1_Z2_Z3_Z4_Z5_summary.pdf
----** compiling Runs/TEST_VMMF/VMMF_FirstOrder_Z1_Z2_Z3_Z4_Z5/VMMF_FirstOrder_Z1_Z2_Z3_Z4_Z5_summary.tex **----
PDF generated: Runs/TEST_VMMF/VMMF_FirstOrder_Z1_Z2_Z3_Z4_Z5/VMMF_FirstOrder_Z1_Z2_Z3_Z4_Z5_summary.pdf
=== Compiling DONE ===



# Automated Search

In [25]:
from   symcircuit.symbolic_solver.solver     import run_experiment

circuit = VMMF.circuit


experiment_result = run_experiment("TEST_VMMF_Automated",
               T_type="NA",
               circuit=circuit,
               minNumOfActiveImpedances=3,
               maxNumOfActiveImpedances=5,
               impedanceKeysOverwrite= None,
               outputFrom= ["V2", "0"],
               inputFrom=["V1", "0"],
               loadHistory=True
               )



--- Found a unique pkl file at Runs/TEST_VMMF_Automated_NA/results_circuit_solution.pkl
--- Updated the circuit solver
Loading the circuit solver from past runs
** Loading the circuit history
Found a unique results.pkl file at Runs/TEST_VMMF_Automated_NA
** Found 25 keys already computed
Performing a systematic search: min_num_of_active_z = 3 max_num_of_active_z = 5
Experiment will be ran for 6 keys: [{}, {'Z1_Z2_Z3_Z4': (Z_1, Z_2, Z_3, Z_4), 'Z1_Z2_Z3_Z5': (Z_1, Z_2, Z_3, Z_5), 'Z1_Z2_Z4_Z5': (Z_1, Z_2, Z_4, Z_5), 'Z1_Z3_Z4_Z5': (Z_1, Z_3, Z_4, Z_5), 'Z2_Z3_Z4_Z5': (Z_2, Z_3, Z_4, Z_5)}, {'Z1_Z2_Z3_Z4_Z5': (Z_1, Z_2, Z_3, Z_4, Z_5)}]
==> Running the TEST_VMMF_Automated_NA Experiment for Z1_Z2_Z3_Z4 (progress: 1/5 combo size: 2/3)

combo key = Z1_Z2_Z3_Z4
Zcombos


Getting the TFs (CG): 100%|██████████| 256/256 [00:00<00:00, 1519.60combo/s]


Number of transfer functions found: 256


Computing Biquad filter parameters: 100%|██████████| 256/256 [00:00<00:00, 2600.62filter/s]


HP : 0
BP : 16
LP : 0
BS : 0
GE : 0
AP : 0
INVALID-NUMER : 0
INVALID-WZ : 0
INVALID-ORDER : 240
PolynomialError : 0
LaTeX report generated and saved to: Runs/TEST_VMMF_Automated_NA/TEST_VMMF_Automated_NA_Z1_Z2_Z3_Z4/TEST_VMMF_Automated_NA_Z1_Z2_Z3_Z4_summary.tex

=== Compiling the reports to PDF ===
----** compiling Runs/TEST_VMMF_Automated_NA/TEST_VMMF_Automated_NA_Z1_Z2_Z3_Z4/TEST_VMMF_Automated_NA_Z1_Z2_Z3_Z4_summary.tex **----
PDF generated: Runs/TEST_VMMF_Automated_NA/TEST_VMMF_Automated_NA_Z1_Z2_Z3_Z4/TEST_VMMF_Automated_NA_Z1_Z2_Z3_Z4_summary.pdf
=== Compiling DONE ===


=== ExperimentResult saved successfully to Runs/TEST_VMMF_Automated_NA/results.pkl - 119.304 kb ===

==> Running the TEST_VMMF_Automated_NA Experiment for Z1_Z2_Z3_Z5 (progress: 2/5 combo size: 2/3)

combo key = Z1_Z2_Z3_Z5
Zcombos


...

HP : 0
BP : 0
LP : 0
BS : 0
GE : 0
AP : 0
INVALID-NUMER : 0
INVALID-WZ : 0
INVALID-ORDER : 256
PolynomialError : 0
!!! INVALID Z_arr !!!

=== Compiling the reports to PDF ===
=== Compiling DONE ===


=== ExperimentResult saved successfully to Runs/TEST_VMMF_Automated_NA/results.pkl - 357.696 kb ===

==> Running the TEST_VMMF_Automated_NA Experiment for Z1_Z2_Z3_Z4_Z5 (progress: 1/1 combo size: 3/3)

combo key = Z1_Z2_Z3_Z4_Z5
Zcombos


Getting the TFs (CG): 100%|██████████| 1024/1024 [00:09<00:00, 102.65combo/s]


Number of transfer functions found: 1024


Computing Biquad filter parameters: 100%|██████████| 1024/1024 [00:18<00:00, 55.67filter/s]


HP : 25
BP : 70
LP : 25
BS : 0
GE : 0
AP : 0
INVALID-NUMER : 192
INVALID-WZ : 86
INVALID-ORDER : 626
PolynomialError : 0
LaTeX report generated and saved to: Runs/TEST_VMMF_Automated_NA/TEST_VMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5/TEST_VMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5_summary.tex

=== Compiling the reports to PDF ===
----** compiling Runs/TEST_VMMF_Automated_NA/TEST_VMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5/TEST_VMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5_summary.tex **----
PDF generated: Runs/TEST_VMMF_Automated_NA/TEST_VMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5/TEST_VMMF_Automated_NA_Z1_Z2_Z3_Z4_Z5_summary.pdf
=== Compiling DONE ===


=== ExperimentResult saved successfully to Runs/TEST_VMMF_Automated_NA/results.pkl - 959.16 kb ===

<----> END OF EXPERIMENT <---->
Impedance Keys analyzed (count: 6): 
[{},
 {'Z1_Z2_Z3_Z4': (Z_1, Z_2, Z_3, Z_4),
  'Z1_Z2_Z3_Z5': (Z_1, Z_2, Z_3, Z_5),
  'Z1_Z2_Z4_Z5': (Z_1, Z_2, Z_4, Z_5),
  'Z1_Z3_Z4_Z5': (Z_1, Z_3, Z_4, Z_5),
  'Z2_Z3_Z4_Z5': (Z_2, Z_3, Z_4, Z_5)},
 {'Z1_Z2_Z3_Z4_Z5': (Z

In [33]:
classifications, tf = experiment_result.get("Z1_Z2_Z3_Z4_Z5")
tf

-Z_2*Z_4*Z_5/(Z_1*Z_2*Z_3 + Z_1*Z_2*Z_4 + Z_1*Z_3*Z_4 + Z_1*Z_4*Z_5 + Z_2*Z_3*Z_4)

In [35]:
type(classifications), type(classifications[-1])

(list, symcircuit.symbolic_solver.domains.Filter_Classification)

In [37]:
df_classifications = experiment_result.flatten_classifications()
df_classifications

,impedance_key,zCombo,transferFunc,transferFunc_latex,valid,fType,parameters,filterOrder
0,Z1,"(R_1, oo, oo, oo, oo)",0,0,False,INVALID-ORDER,None,BiQuad
1,Z1,"(1/(C_1*s), oo, oo, oo, oo)",0,0,False,INVALID-ORDER,None,BiQuad
2,Z1,"(R_1 + 1/(C_1*s), oo, oo, oo, oo)",0,0,False,INVALID-ORDER,None,BiQuad
3,Z1,"(R_1/(C_1*R_1*s + 1), oo, oo, oo, oo)",0,0,False,INVALID-ORDER,None,BiQuad
4,Z2,"(oo, R_2, oo, oo, oo)",0,0,False,INVALID-ORDER,None,BiQuad
...,...,...,...,...,...,...,...,...
3119,Z1_Z2_Z3_Z4_Z5,"(R_1/(C_1*R_1*s + 1), R_2/(C_2*R_2*s + 1), R_3...",(-C_1*C_3*C_4*R_1*R_2*R_3*R_4*R_5*s**3 - R_2*R...,\frac{- C_{1} C_{3} C_{4} R_{1} R_{2} R_{3} R_...,False,INVALID-ORDER,None,BiQuad
3120,Z1_Z2_Z3_Z4_Z5,"(R_1/(C_1*R_1*s + 1), R_2/(C_2*R_2*s + 1), R_3...",(-C_1*C_3*R_1*R_2*R_3*R_4*R_5*s**2 - R_2*R_4*R...,\frac{- C_{1} C_{3} R_{1} R_{2} R_{3} R_{4} R_...,False,INVALID-WZ,{'Q': C_2*C_3*R_2*R_3*R_5*sqrt(1/(C_2*C_3*R_4*...,BiQuad
3121,Z1_Z2_Z3_Z4_Z5,"(R_1/(C_1*R_1*s + 1), R_2/(C_2*R_2*s + 1), R_3...",(-C_1*C_3*R_1*R_2*R_3*R_4*s**2 - R_2*R_4 + s*(...,\frac{- C_{1} C_{3} R_{1} R_{2} R_{3} R_{4} s^...,False,INVALID-WZ,{'Q': (C_1*C_5*R_1*R_2*R_3*R_4*sqrt(1/(C_1*C_5...,BiQuad
3122,Z1_Z2_Z3_Z4_Z5,"(R_1/(C_1*R_1*s + 1), R_2/(C_2*R_2*s + 1), R_3...",(-C_1*C_3*C_5*R_1*R_2*R_3*R_4*R_5*s**3 - R_2*R...,\frac{- C_{1} C_{3} C_{5} R_{1} R_{2} R_{3} R_...,False,INVALID-ORDER,None,BiQuad


In [38]:
df_tfs = experiment_result.flatten_tfs()
df_tfs

,impedance_key,transferFunc,transferFunc_latex
0,Z1,0,0
1,Z2,0,0
2,Z3,0,0
3,Z4,0,0
4,Z5,0,0
5,Z1_Z2,0,0
6,Z1_Z3,-oo*sign(1/(Z_1 + Z_3)),- \infty \operatorname{sign}{\left(\frac{1}{Z_...
7,Z1_Z4,0,0
8,Z1_Z5,0,0
9,Z2_Z3,0,0


In [43]:
experiment_result.to_csv()

Writting TEST_VMMF_Automated_NA to csv files
flattened all the classifications to Runs/TEST_VMMF_Automated_NA/classifications.csv
flattened all the classifications to Runs/TEST_VMMF_Automated_NA/tfs.csv
